In [ ]:
import os
import sys

# Clone or update the repository locally
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/kaggle/working/6D_pose"   # Adjust this path for your environment
branch = "main"

# Clone the repo if it does not exist; otherwise hard-reset to the remote branch
if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url} to {repo_dir}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url} to {repo_dir}")

# Add the repository to the Python import path
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

In [3]:
import sys

sys.path.append("../..")

In [ ]:
!pip install -U ultralytics
from ultralytics import YOLO
from src.detection.yolo_utils import calculate_adapted_map50, visualize_bbox, create_teacher_dataset_final, load_test_dataset_from_disk

In [ ]:
# Generate a synthetic training dataset (collage images)
import os
import cv2
import numpy as np
import random
import yaml
import shutil
import requests
from tqdm import tqdm

# Configuration
SOURCE_ROOT = '/kaggle/input/line-mode/Linemod_preprocessed/data'
DEST_ROOT = '/kaggle/working/datasets'
BG_CACHE_DIR = '/kaggle/working/backgrounds_cache'

NUM_COLLAGES = 4000      # Number of synthetic collage images to generate
MAX_OBJECTS_PER_IMG = 7  # Maximum objects per collage image

train_subset, test_subset = create_teacher_dataset_final(SOURCE_ROOT,DEST_ROOT,BG_CACHE_DIR,NUM_COLLAGES,MAX_OBJECTS_PER_IMG)

In [ ]:
# Train a YOLO model on the synthetic dataset
import torch

# Hardware check
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ WARNING: Using CPU. Enable a GPU runtime if available.")

# Model selection
model = YOLO('yolo11s.pt')

print("Starting training...")

results = model.train(
    data='/kaggle/working/linemod.yaml',

    # Training schedule
    epochs=60,
    patience=15,

    # Hardware / performance
    imgsz=640,
    batch=64,            # Adjust down if you hit OOM
    device=[0, 1],
    workers=6,
    amp=True,            # Mixed precision

    # Augmentation (the data is already collaged, so keep mosaic/mixup moderate)
    mosaic=0.5,
    mixup=0.0,

    degrees=10.0,
    scale=0.5,
    translate=0.1,
    fliplr=0.5,

    # Color / lighting jitter
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,

    close_mosaic=10,

    # Output
    project='/kaggle/working/YOLO_Runs',
    name='yolo11s_collage_run',
    verbose=True,
    exist_ok=True,
    save=True
)

In [ ]:
# Auto-labeling: use the trained model to add missing labels
import os
import cv2
import numpy as np
import yaml
import shutil
from ultralytics import YOLO
from tqdm import tqdm

DEST_ROOT = '/kaggle/working/dataset_AUTOLABELED'
MODEL_PATH = '/kaggle/working/YOLO_Runs/yolo11s_collage_run/weights/best.pt'

create_student_dataset_final(DEST_ROOT, MODEL_PATH, train_subset,'/kaggle/working/datasets')

In [ ]:
# Final training on the auto-labeled dataset
from ultralytics import YOLO

model = YOLO('yolo11s.pt')

results = model.train(
    data='/kaggle/working/dataset_AUTOLABELED/student.yaml',

    # Training schedule
    epochs=100,
    patience=15,

    # Hardware / performance
    imgsz=640,
    batch=64,            # Adjust down if you hit OOM
    device=[0, 1],
    workers=6,

    # Augmentation
    mosaic=1.0,
    mixup=0.1,

    degrees=10.0,
    fliplr=0.5,
    scale=0.5,

    # Output
    project='/kaggle/working/YOLO_Runs',
    name='yolo11s_autolabel_final_with_80_th',
    save=True,
    exist_ok=True
)

In [ ]:
save_test_dataset_to_disk(test_subset)

In [ ]:
calculate_adapted_map50('/content/drive/MyDrive/YOLO_Runs/yolo11s_autolabel_final/weights/best.pt',
                        '/content/data/Linemod_preprocessed/data')